In [1]:
cd ..

/Users/zfac230/botanica


/Users/zfac230/.venv/botanica/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
!pwd

/Users/zfac230/botanica


In [3]:
import torch
import torch.nn as nn
from torchvision import models 
from torchinfo import summary
from torchvision import transforms as T

from data.datasets import BinarySiameseImageFolder
from transformer.architecture import SiameseFewShot, SiameseTowers
from transformer.utils import triplet_train_model

/Users/zfac230/.venv/botanica/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [5]:
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224
IMAGE_SIZE = (IMAGE_HEIGHT, IMAGE_WIDTH)
EPOCHS = 10 #"Don't forget to define me!"
train_dir = "toy_data/"
transforms = T.Compose([
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
            T.RandomApply(torch.nn.ModuleList([T.ColorJitter()]), p=0.25),
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.RandomErasing(p=0.1, value='random')
        ])
trainset = BinarySiameseImageFolder(train_dir, transform = transforms)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=3, num_workers=1)

valset = BinarySiameseImageFolder(train_dir, transform = transforms)
valloader = torch.utils.data.DataLoader(trainset, batch_size=3, num_workers=1)

In [6]:
encoder = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
dist_func = torch.cdist

In [7]:
support_set1, support_set2 = trainset.get_support_set(5)

In [8]:
len(support_set1)

5

In [9]:
model = SiameseFewShot(encoder, dist_func)

In [10]:
item, _, _, target = trainset.__getitem__(0)

In [11]:
len(item)

3

In [12]:
target

0

In [13]:
classes = trainset.classes
from tqdm import tqdm
EPOCHS = 12

tower = models.vit_l_16(weights = models.ViT_L_16_Weights.IMAGENET1K_SWAG_LINEAR_V1)

for param in tower.parameters():
    param.requires_grad = False
    
tower.heads.head = nn.Identity()

In [14]:
tower

VisionTransformer(
  (conv_proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=1024, out_features=4096, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=4096, out_features=1024, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
       

In [15]:
siamese_model = SiameseTowers(tower).to(mps_device)
for param in siamese_model.tower.encoder.layers.encoder_layer_23.parameters():
    param.requires_grad = True
for param in siamese_model.tower.encoder.ln.parameters():
    param.requires_grad = True
triplet_loss = nn.TripletMarginLoss(margin=1.0, p=2, eps=1e-7)

In [16]:
def dist_func(a,b): 
    return torch.abs(b - a)

In [ ]:
params = list(siamese_model.tower.encoder.layers.encoder_layer_23.parameters()) + list(siamese_model.tower.encoder.ln.parameters())
optimizer = torch.optim.AdamW(params)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10)
siamese_model = triplet_train_model(siamese_model, triplet_loss, optimizer, exp_lr_scheduler, trainloader, trainset, valloader, valset, mps_device, dist_func, num_epochs=10)

Epoch 0/9
----------
